`Aim:` *Notebook for the purpose of Climate Risk Assessment lecture:* `Profiling code`

`Author:` Martin Aregger

This Notebook contains a number of simple examples on how you can profile your python content. It is by far not exhaustive and you can find many more tools online!

# Profiling the timing of code 

Examples of 4 different ways to measure the time your code takes. Each of them does something a bit different, so they may be complement each other.

- **time.time()**: Measures wall-clock time.
- **timeit.timeit()**: Repeats the execution multiple times for averaging.
- **cProfile**: Reports function-level timing and call counts.
- **line_profiler**: Provides line-by-line profiling for in-depth analysis.

In [ ]:
import time
import timeit
import cProfile
from line_profiler import LineProfiler
# this is a third party module, install it in your environment! is a third party module, install it in your environment!

import numpy as np

In [ ]:
# Helper functions
def generate_matrix(size):
    """Generates a random matrix."""
    return np.random.rand(size, size)


def multiply_matrices(A, B):
    """Performs matrix multiplication."""
    return np.dot(A, B)


def compute_statistics(matrix):
    """Computes some statistics on a matrix."""
    mean = np.mean(matrix)
    std = np.std(matrix)
    max_val = np.max(matrix)
    min_val = np.min(matrix)
    return {"mean": mean, "std": std, "max": max_val, "min": min_val}


# Example function to profile
def example_function():
    # Main computation
    size = 2000
    iterations = 5
    stats = []
    for _ in range(iterations):
        A = generate_matrix(size)
        B = generate_matrix(size)
        result = multiply_matrices(A, B)
        stats.append(compute_statistics(result))
    return


In [3]:
# simplest way to profile a function --> use the timer of the jupyter notebook cell...
example_function()

# Very inconsistent!

In [8]:
# Method 1: Using time.time()
start_time = time.time()
example_function()
end_time = time.time()
print(f"[time.time] Execution time: {end_time - start_time:.5f} seconds")

# This method measures the time between the start time and end time
# It is called wall time, and it includes the time spent by the process waiting for the CPU, I/O operations, etc.

[time.time] Execution time: 1.10023 seconds


In [9]:
# Method 2: Using timeit.timeit()
execution_time = timeit.timeit("example_function()", globals=globals(), number=5)
print(f"[timeit] Average execution time over 5 runs: {execution_time / 5:.5f} seconds")

# This method is more accurate than time.time() because it runs the function multiple times and calculates the average time
# It is useful when the function execution time is short

[timeit] Average execution time over 5 runs: 1.06003 seconds


In [10]:
# Method 3: Using cProfile
cProfile.run("example_function()")

# This method provides detailed information about the function execution time, number of calls, and more

         309 function calls in 1.067 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.149    0.030 1948630586.py:10(compute_statistics)
       10    0.001    0.000    0.413    0.041 1948630586.py:2(generate_matrix)
        1    0.000    0.000    1.065    1.065 1948630586.py:22(example_function)
        5    0.503    0.101    0.503    0.101 1948630586.py:6(multiply_matrices)
        1    0.002    0.002    1.066    1.066 <string>:1(<module>)
        5    0.004    0.001    0.033    0.007 _methods.py:101(_mean)
        5    0.054    0.011    0.082    0.016 _methods.py:135(_var)
        5    0.002    0.000    0.084    0.017 _methods.py:204(_std)
       10    0.001    0.000    0.001    0.000 _methods.py:67(_count_reduce_items)
       10    0.000    0.000    0.000    0.000 _ufunc_config.py:452(_no_nep50_warning)
        5    0.000    0.000    0.000    0.000 contextlib.py:102(__init__)
        5    0

In [11]:
# Method 4: Using line_profiler
profiler = LineProfiler()
profiler.add_function(example_function)
profiler.run("example_function()")
profiler.print_stats()

#

Timer unit: 1e-09 s

Total time: 1.08528 s
File: /tmp/ipykernel_117215/1948630586.py
Function: example_function at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def example_function():
    23                                           
    24                                           
    25                                               # Main computation
    26         1        841.0    841.0      0.0      size = 2000
    27         1        160.0    160.0      0.0      iterations = 5
    28         1        160.0    160.0      0.0      stats = []
    29         6     404199.0  67366.5      0.0      for _ in range(iterations):
    30         5  204991699.0    4e+07     18.9          A = generate_matrix(size)
    31         5  272654606.0    5e+07     25.1          B = generate_matrix(size)
    32         5  454339408.0    9e+07     41.9          result = multiply_matrices(A, B)
    33         5  152888578.0    3

In [ ]:
# profile all the nested functions in the example_function with line_profiler

profiler = LineProfiler()
profiler.add_function(example_function)
profiler.add_function(generate_matrix)
profiler.add_function(multiply_matrices)
profiler.add_function(compute_statistics)
profiler.run("example_function()")
profiler.print_stats()


Timer unit: 1e-09 s

Total time: 0.55558 s
File: /tmp/ipykernel_117215/1948630586.py
Function: generate_matrix at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def generate_matrix(size):
     3                                               """Generates a random matrix."""
     4        10  555579934.0    6e+07    100.0      return np.random.rand(size, size)

Total time: 0.504072 s
File: /tmp/ipykernel_117215/1948630586.py
Function: multiply_matrices at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def multiply_matrices(A, B):
     7                                               """Performs matrix multiplication."""
     8         5  504071893.0    1e+08    100.0      return np.dot(A, B)

Total time: 0.177147 s
File: /tmp/ipykernel_117215/1948630586.py
Function: compute_statistics at line 10

Line #      Hits         Time  Per Hit   % Time  

## Jupyter notebook specific profiling

Jupyter has a number of inbuilt magic commands that can be used to profile code. They are:
- **%time**: Time the execution of a single statement
- **%timeit**: Time repeated execution of a single statement for more accuracy
- **%prun**: Run code with the profiler inbuilt in Jupyter
- **%lprun**: Run code with the line-by-line profiler

In [13]:
%time example_function()

CPU times: user 45.5 s, sys: 1min 10s, total: 1min 56s
Wall time: 1.03 s


In [14]:
%timeit example_function()

910 ms ± 6.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%prun example_function()

         309 function calls in 0.919 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.416    0.083    0.416    0.083 1948630586.py:6(multiply_matrices)
       10    0.347    0.035    0.347    0.035 {method 'rand' of 'numpy.random.mtrand.RandomState' objects}
       25    0.103    0.004    0.103    0.004 {method 'reduce' of 'numpy.ufunc' objects}
        5    0.047    0.009    0.073    0.015 _methods.py:135(_var)
        1    0.002    0.002    0.918    0.918 <string>:1(<module>)
       10    0.001    0.000    0.001    0.000 _methods.py:67(_count_reduce_items)
       10    0.001    0.000    0.348    0.035 1948630586.py:2(generate_matrix)
        1    0.001    0.001    0.919    0.919 {built-in method builtins.exec}
        5    0.000    0.000    0.073    0.015 _methods.py:204(_std)
        5    0.000    0.000    0.152    0.030 1948630586.py:10(compute_statistics)
        5    0.000    0.000    0.047    0.009 _met

In [ ]:
# alternative way to call line profiler in jupyter notebook

%load_ext line_profiler
%lprun -f example_function example_function()

Timer unit: 1e-09 s

Total time: 1.10195 s
File: /tmp/ipykernel_117215/1948630586.py
Function: example_function at line 22

Line #      Hits         Time  Per Hit   % Time  Line Contents
    22                                           def example_function():
    23                                           
    24                                           
    25                                               # Main computation
    26         1        962.0    962.0      0.0      size = 2000
    27         1        150.0    150.0      0.0      iterations = 5
    28         1        190.0    190.0      0.0      stats = []
    29         6     531844.0  88640.7      0.0      for _ in range(iterations):
    30         5  235658396.0    5e+07     21.4          A = generate_matrix(size)
    31         5  326289294.0    7e+07     29.6          B = generate_matrix(size)
    32         5  403405517.0    8e+07     36.6          result = multiply_matrices(A, B)
    33         5  136064089.0    3

In [17]:
# calling all the nested functions in the example_function
%lprun -f example_function -f generate_matrix -f multiply_matrices -f compute_statistics example_function()

Timer unit: 1e-09 s

Total time: 0.52819 s
File: /tmp/ipykernel_117215/1948630586.py
Function: generate_matrix at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def generate_matrix(size):
     3                                               """Generates a random matrix."""
     4        10  528189746.0    5e+07    100.0      return np.random.rand(size, size)

Total time: 0.423728 s
File: /tmp/ipykernel_117215/1948630586.py
Function: multiply_matrices at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def multiply_matrices(A, B):
     7                                               """Performs matrix multiplication."""
     8         5  423727738.0    8e+07    100.0      return np.dot(A, B)

Total time: 0.200685 s
File: /tmp/ipykernel_117215/1948630586.py
Function: compute_statistics at line 10

Line #      Hits         Time  Per Hit   % Time  

## Profiling a script

You can profile any script directly in the terminal using simple commmands.

In [ ]:
%%file example_module.py
# this is a magic command to save the content of the cell to a file
# we create a module from the example_function which we can then profile

import numpy as np


# Helper functions
def generate_matrix(size):
    """Generates a random matrix."""
    return np.random.rand(size, size)


def multiply_matrices(A, B):
    """Performs matrix multiplication."""
    return np.dot(A, B)


def compute_statistics(matrix):
    """Computes some statistics on a matrix."""
    mean = np.mean(matrix)
    std = np.std(matrix)
    max_val = np.max(matrix)
    min_val = np.min(matrix)
    return {"mean": mean, "std": std, "max": max_val, "min": min_val}


# Example function to profile


def example_function():
    # Main computation
    size = 2000
    iterations = 5
    stats = []
    for _ in range(iterations):
        A = generate_matrix(size)
        B = generate_matrix(size)
        result = multiply_matrices(A, B)
        stats.append(compute_statistics(result))
    return


if __name__ == "__main__":
    example_function()


Writing example_module.py


In [19]:
# profile with
!kernprof -l example_module.py 
# you can run this command in your command line directly without the !

Wrote profile results to example_module.py.lprof
Inspect results with:
python -m line_profiler -rmt "example_module.py.lprof"


In [ ]:
# look at output with:
!python -m line_profiler example_module.py.lprof
# to look at the output

Timer unit: 1e-06 s



The output is empty!


**ANY FUNCTION YOU WANT TO PROFILE MUST BE ANNOTATED WITH THE @profile decorator**

... don't forget to remove the decorators once your done with profiling.

In [ ]:
%%file example_module.py
# this is a magic command to save the content of the cell to a file
# we create a module from the example_function which we can then profile

import numpy as np


# Helper functions
def generate_matrix(size):
    """Generates a random matrix."""
    return np.random.rand(size, size)


def multiply_matrices(A, B):
    """Performs matrix multiplication."""
    return np.dot(A, B)


@profile
def compute_statistics(matrix):
    """Computes some statistics on a matrix."""
    mean = np.mean(matrix)
    std = np.std(matrix)
    max_val = np.max(matrix)
    min_val = np.min(matrix)
    return {"mean": mean, "std": std, "max": max_val, "min": min_val}


# Example function to profile


@profile  # HERE
def example_function():
    # Main computation
    size = 2000
    iterations = 5
    stats = []
    for _ in range(iterations):
        A = generate_matrix(size)
        B = generate_matrix(size)
        result = multiply_matrices(A, B)
        stats.append(compute_statistics(result))
    return


if __name__ == "__main__":
    example_function()


Overwriting example_module.py


In [25]:
!kernprof -l -v example_module.py 
# !python -m line_profiler example_module.py.lprof

Wrote profile results to example_module.py.lprof
Timer unit: 1e-06 s

Total time: 0.168636 s
File: example_module.py
Function: compute_statistics at line 15

Line #      Hits         Time  Per Hit   % Time  Line Contents
    15                                           @profile
    16                                           def compute_statistics(matrix):
    17                                               """Computes some statistics on a matrix."""
    18         5      57405.4  11481.1     34.0      mean = np.mean(matrix)
    19         5      84313.0  16862.6     50.0      std = np.std(matrix)
    20         5      14084.0   2816.8      8.4      max_val = np.max(matrix)
    21         5      12824.8   2565.0      7.6      min_val = np.min(matrix)
    22         5          9.3      1.9      0.0      return {"mean": mean, "std": std, "max": max_val, "min": min_val}

Total time: 1.08313 s
File: example_module.py
Function: example_function at line 29

Line #      Hits         Time  P

# Memory Profiling

To profile the memory usage of the function, you can use memory_profiler, also a third party package that you may install with pip
 

In [26]:
%load_ext memory_profiler

# Profile overall memory usage
%memit example_function()

peak memory: 493.80 MiB, increment: 126.14 MiB


In [27]:
%%memit # profil a block of code
size = 2000
iterations = 5
A = np.random.rand(size, size)
B = np.random.rand(size, size)
result = np.dot(A, B)

peak memory: 494.01 MiB, increment: 121.87 MiB


Like **lprun** with have **mprun** for memory profiling
however, it only works with functions defined in separate files, not in the notebook so lets create a file with a function and profile it


In [ ]:
%%file mprun_demo.py
# this cell magic writes the content of the cell to a file

import numpy as np


def matrix_factorization_example():
    """Performs matrix factorization and manipulation to showcase memory usage"""
    size = 1500  # Use a large matrix size for better profiling visibility
    num_iterations = 3

    # Step 1: Initialize matrices
    A = np.random.rand(size, size)  # Random matrix
    B = np.random.rand(size, size)  # Random matrix
    print(f"Initial matrices created: {A.shape} and {B.shape}")

    # Step 2: Matrix multiplication and memory manipulation
    for _ in range(num_iterations):
        C = np.dot(A, B)  # Multiply matrices

        # Step 3: Modify the result with deep copy and reshape operations
        C_copy = C.copy()  # Deep copy to simulate an expensive operation
        C_reshaped = C_copy.reshape(size * size)  # Flattening for manipulation
        C_reshaped = C_reshaped + np.random.rand(size * size)  # Add random noise

        # Step 4: Perform a computation that involves slicing and reshaping
        C_sliced = C_reshaped[: int(size * size / 2)]  # Slice first half

        # Step 5: Create a new matrix and add it to the original
        D = np.random.rand(size, size)
        C_result = (
            C_sliced.reshape(size, size // 2) + D[:, : size // 2]
        )  # Element-wise addition

    # Final result
    return C_result


if __name__ == "__main__":
    matrix_factorization_example()

Writing mprun_demo.py


In [ ]:
from mprun_demo import (
    matrix_factorization_example,
)  # now we can import our function and profile it

%mprun -f matrix_factorization_example matrix_factorization_example()

Initial matrices created: (1500, 1500) and (1500, 1500)



Filename: /users/maregger/PhD/Python_Tutorial/mprun_demo.py

Line #    Mem usage    Increment  Occurences   Line Contents
     6    492.9 MiB    492.9 MiB           1   def matrix_factorization_example():
     7                                             """Performs matrix factorization and manipulation to showcase memory usage"""
     8    492.9 MiB      0.0 MiB           1       size = 1500  # Use a large matrix size for better profiling visibility
     9    492.9 MiB      0.0 MiB           1       num_iterations = 3
    10                                             
    11                                             # Step 1: Initialize matrices
    12    493.1 MiB      0.1 MiB           1       A = np.random.rand(size, size)  # Random matrix
    13    510.1 MiB     17.1 MiB           1       B = np.random.rand(size, size)  # Random matrix
    14    510.1 MiB      0.0 MiB           1       print(f"Initial matrices created: {A.shape} and {B.shape}")
    15                          

In [ ]:
# look at tracemalloc
import tracemalloc

# Start tracing memory allocations
tracemalloc.start()

matrix_factorization_example()

# Take a snapshot
snapshot = tracemalloc.take_snapshot()

# create a large matrix that will use lots of ram
large_matrix = np.random.rand(10000, 10000)

# Take a second snapshot
snapshot2 = tracemalloc.take_snapshot()


# Analyze top memory consumers in snapshot 1
top_stats = snapshot.statistics("lineno")
print("[ Top 10 memory consumers ]")
for stat in top_stats[:10]:
    print(stat)

# Stop tracing memory allocations
tracemalloc.stop()

# careful when using jupyter - it might use most of the memory.

Initial matrices created: (1500, 1500) and (1500, 1500)
[ Top 10 memory consumers ]
/users/maregger/micromamba/envs/hailclass_exp/lib/python3.10/codeop.py:118: size=448 B, count=6, average=75 B
/users/maregger/micromamba/envs/hailclass_exp/lib/python3.10/site-packages/zmq/sugar/attrsettr.py:44: size=330 B, count=6, average=55 B
/users/maregger/micromamba/envs/hailclass_exp/lib/python3.10/site-packages/traitlets/traitlets.py:731: size=209 B, count=3, average=70 B
/users/maregger/micromamba/envs/hailclass_exp/lib/python3.10/site-packages/traitlets/traitlets.py:1514: size=168 B, count=1, average=168 B
/users/maregger/micromamba/envs/hailclass_exp/lib/python3.10/site-packages/ipykernel/iostream.py:286: size=168 B, count=1, average=168 B
/users/maregger/micromamba/envs/hailclass_exp/lib/python3.10/site-packages/ipykernel/iostream.py:689: size=144 B, count=2, average=72 B
/users/maregger/PhD/Python_Tutorial/mprun_demo.py:30: size=120 B, count=3, average=40 B
/users/maregger/micromamba/envs/h

In [ ]:
# Compare the snapshots
stats1_vs_2 = snapshot2.compare_to(snapshot, "lineno")

# Save the comparison results to a text file
with open("memory_leak_analysis.txt", "w") as f:
    f.write("[ Memory usage increase from snapshot 1 to snapshot 2 ]\n")
    for stat in stats1_vs_2[:10]:
        f.write(f"{stat}\n")

    # Detailed traceback for the top memory consumers
    f.write("\n[ Detailed traceback for the top memory consumers ]\n")
    for stat in stats1_vs_2[:1]:
        f.write("\n".join(stat.traceback.format()) + "\n")

print("Memory leak analysis saved to 'memory_leak_analysis.txt'")

Memory leak analysis saved to 'memory_leak_analysis.txt'


**Careful when using dask! It waits with executing any computation until you call compute() or save your output. This means all the resources will be used there...**